In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Retriever Implementation**

In [3]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 58.0 MB/s eta 0:00:00


In [4]:
import faiss
index_path = "/content/drive/MyDrive/Tenx/Week 6/faiss_index.idx"
index = faiss.read_index(index_path)

In [2]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7db8cc4fb300> >

In [5]:
import pandas as pd
df= pd.read_csv("/content/drive/MyDrive/Tenx/Week 6/faiss_metadata.csv")

In [ ]:
df

,Complaint ID,Product,chunks
0,14069121,Credit card,['a card was opened under my name by a fraudst...
1,14047085,Credit card,['i have a secured credit card with citibank w...
2,14040217,Credit card,['i have a citi rewards cards the credit balan...
3,13968411,Credit card,['bi am writing to dispute the following charg...
4,13965746,Credit card,['although the account had been deemed closed ...
...,...,...,...
319341,5638610,"Money transfer, virtual currency, or money ser...",['i moved into owned and operated by on each m...
319342,5896172,"Payday loan, title loan, or personal loan",['omni financial loan paid for by allotment']
319343,5896060,"Money transfer, virtual currency, or money ser...",['hello im an of company that from and to \ni ...
319344,6238646,Checking or savings account,['i already have a case but it should not have...


In [30]:
import RAG
import importlib
importlib.reload(RAG)

<module 'RAG' from '/content/RAG.py'>

In [8]:
#take a user's question
query=RAG.get_user_query()

Ask your question: What are the major complaints about saving accounts?


In [11]:
#Embed the question
model = RAG.load_embedding_model()
embedding = model.encode([query])

In [12]:
embedding

array([[-9.24077001e-04,  7.03098252e-02, -5.12801185e-02,
         2.44758911e-02,  1.28997695e-02,  7.61032254e-02,
         5.37804840e-03, -5.49689028e-03,  1.75731108e-02,
         6.45922944e-02,  8.40006955e-03,  3.59292468e-03,
         9.80375614e-03, -5.80706894e-02, -2.39409730e-02,
        -6.10014610e-02, -4.72978055e-02, -2.97565144e-02,
        -5.69060333e-02,  2.28799414e-02, -4.82594371e-02,
        -5.72920181e-02,  6.24171458e-03,  8.23475271e-02,
         2.10532639e-02, -1.82607081e-02,  3.15568149e-02,
        -1.93835292e-02, -4.48682867e-02, -4.06429619e-02,
        -2.08041258e-02,  1.32000260e-02,  2.21021902e-02,
        -5.15544675e-02, -1.91534422e-02,  4.34575379e-02,
        -1.01331891e-02,  9.35152620e-02, -3.27745453e-02,
        -1.03794307e-01, -1.96529571e-02, -8.19577128e-02,
         7.20981648e-03, -3.35507207e-02,  1.57873263e-04,
         4.68030721e-02,  4.66233231e-02,  1.61339417e-02,
        -4.22862954e-02,  5.51768430e-02,  4.81182337e-0

In [22]:
#Similarity search
result=RAG.similarity_search(index,embedding,df["chunks"],5)

In [23]:
result

["['bank unlawfully misled consumers about its accounts and obscured its higherinterest savings product from them']",
 "['there have been scams associated with my saving account that is from within email addresses and other people personal data are being stolen from people s application they are not doing a good job protecting data']",
 "['personal information being fraudulently used to collect credit vouchers and make purchases']",
 "['i have numerous capital one 360 savings accounts for which deceptive interest rate practices occurred']",
 "['i am filing a complaint against capital one regarding the deceptive practices related to their 360 high yield interest savings account capital one misrepresented the 360 savings accounts']"]

In [24]:
context = "\n\n".join(result)

**Generator Implementation**

In [25]:
#Combine the prompt, the user question, and the retrieved chunks
final_prompt=RAG.build_rag_prompt(query,context)
final_prompt

" You are a helpful and objective financial data analyst.\n You will be given a user's question and a set of context documents retrieved from internal company data. Your task is to answer the question based **only on the information provided in the context**. \n Do not use any prior knowledge or make assumptions that are not grounded in the context. If the context does not contain enough information to answer the question, say so clearly.\n Be concise, clear, and insightful in your response.\n ---\n Context: ['bank unlawfully misled consumers about its accounts and obscured its higherinterest savings product from them']\n\n['there have been scams associated with my saving account that is from within email addresses and other people personal data are being stolen from people s application they are not doing a good job protecting data']\n\n['personal information being fraudulently used to collect credit vouchers and make purchases']\n\n['i have numerous capital one 360 savings accounts f

In [17]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00


In [18]:
!pip install transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 27.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [26]:
#Return the LLM's generated response
response=RAG.llm_usage(final_prompt)

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Device set to use cpu


In [27]:
RAG.extract_answer(response)

'Capital one savings account customers filed complaints against the company and their sub-brand capital one bank, alleging that they misrepresented savings accounts, offering high interest rates without providing information on the true interest rate, and not informing consumers of the terms of the accounts.'

**Reranking for better model**

In [28]:
!pip install tqdm

In [31]:
ranked = RAG.rerank_chunks(query, df["chunks"], top_k=5,batch_size=8)

Re-ranking:   0%|          | 17/39919 [00:39<25:40:29,  2.32s/it]


KeyboardInterrupt: 

In [14]:
context = "\n\n".join(ranked)
final=RAG.build_rag_prompt(query,context)

In [15]:
response2=RAG.llm_usage(final)

Device set to use cuda:0


In [16]:
RAG.extract_answer(response2)

'1. Limitations on withdrawing funds\n 2. Redundant fees and interest rates\n 3. Limited banking options\n 4. Lack of control over funds\n 5. Poor customer service\n 6. Lack of ability to transfer funds across accounts\n 7. Inability to link bank accounts\n 8. Lack of convenience and simplicity\n 9. Limited account options\n 10. Confusion over account activity and account information'

In [32]:
query1= 'what are the major complaints about savings accounts?'
query2= 'what are the major complaints about credit cards?'
query3= 'what are the major complaints about personal loans?'
query4= 'what are the major complaints about buy now pay later?'
query5= 'what are the major complaints about money transfers?'
query6= 'what do people say is the one thing they want to be improved in money transfer?'

In [33]:
embedding1 = model.encode([query1])
embedding2 = model.encode([query2])
embedding3 = model.encode([query3])
embedding4 = model.encode([query4])
embedding5 = model.encode([query5])
embedding6 = model.encode([query6])

In [35]:
result1=RAG.similarity_search(index,embedding1,df["chunks"],5)
result2=RAG.similarity_search(index,embedding2,df["chunks"],5)
result3=RAG.similarity_search(index,embedding3,df["chunks"],5)
result4=RAG.similarity_search(index,embedding4,df["chunks"],5)
result5=RAG.similarity_search(index,embedding5,df["chunks"],5)
result6=RAG.similarity_search(index,embedding6,df["chunks"],5)
context1 = "\n\n".join(result1)
context2 = "\n\n".join(result2)
context3 = "\n\n".join(result3)
context4 = "\n\n".join(result4)
context5 = "\n\n".join(result5)
context6 = "\n\n".join(result6)


In [37]:
final1=RAG.build_rag_prompt(query1,context1)
final2=RAG.build_rag_prompt(query2,context2)
final3=RAG.build_rag_prompt(query3,context3)
final4=RAG.build_rag_prompt(query4,context4)
final5=RAG.build_rag_prompt(query5,context5)
final6=RAG.build_rag_prompt(query6,context6)

In [38]:
response1=RAG.llm_usage(final1)
RAG.extract_answer(response1)

Device set to use cpu


"The major complaints about savings accounts are as follows:\n 1. Misleading interest rates and promises\n 2. Lack of transparency and disclosure\n 3. Failure to fulfill promises\n 4. Lack of competition in the industry\n 5. Negative impact on financial wellbeing\n 6. Abuse of power by financial institutions\n 7. Poor customer service\n 8. High fees and hidden charges\n 9. Difficulty in accessing funds\n 10. Loss of funds due to financial mishaps\n 11. Inability to withdraw funds\n 12. Limited flexibility in account management\n 13. Negligent management of accounts\n 14. Poor customer support\n 15. Lack of accountability and responsibility\n 16. Inability to compare savings accounts\n 17. Inability to open accounts through online channels\n 18. Inability to track account balances\n 19. Inability to transfer funds between accounts\n 20. Lack of options for investment\n 21. Lack of transparency and disclosure in the fees charged\n 22. Lack of security measures to prevent fraud\n 23. Lack

In [39]:
response2=RAG.llm_usage(final2)
RAG.extract_answer(response2)

Device set to use cpu


'major complaints about credit cards include:\n\n1. Incorrectly accruing interest rates\n2. Poor customer service\n3. High credit card fees\n4. Limited credit limits\n5. Late fees for billing errors\n6. Higher interest rates when borrowing through credit cards\n7. High interest rates on cash advances\n8. Fees and fees for credit card transactions\n9. Poor customer service and lack of transparency in charges\n10. Limited ability to switch credit card providers.'

In [40]:
response3=RAG.llm_usage(final3)
RAG.extract_answer(response3)

Device set to use cpu


"Yes, Ascend Loans offers options for people who have missed loan payments. Here's how it works:\n 1. If a borrower has missed loan payments due to an unforeseen circumstance, they can work with their lender to adjust the loan terms, such as extending the repayment period or reducing the interest rate.\n 2. If a borrower's situation has deteriorated, they can also request a loan modification. Loan modifications allow borrowers to negotiate the terms of their loan with their lender.\n 3. If a borrower is unable to pay the loan, they may be able to request a loan deferral. A deferral allows borrowers to take a break from making loan payments until they are able to make a payment.\n 4. If a borrower is experiencing a financial crisis, they may be able to request a loan forgiveness program. A loan forgiveness program allows borrowers to have their loan forgiven after they make a certain number of payments.\n 5. Ascend Loans also offers debt consolidation services that can help borrowers co

In [41]:
response4=RAG.llm_usage(final4)
RAG.extract_answer(response4)

Device set to use cpu


"'companies need to be more transparent with their interest and fees for products they are marketing as buynow paylater'\n ---"

In [42]:
response5=RAG.llm_usage(final5)
RAG.extract_answer(response5)

Device set to use cpu


'A. Two money transfers were sent via a money transfer to a bank account overseas without disclosing the transaction was fraudulent\n B. Junk fees on international money transfers\n C. The money never arrived to the intended recipients\n D. The money transfers were made without proper warning from financial institution\n E. There was no disclaimer on money transfers made to avoid proper warning from financial institution\n F. Money transfers were sent via money transfer to a bank account overseas\n G. Money transfers were made without proper warning from financial institution\n H. The money transfers were made without proper notice to customers\n I. The money transfers were made without proper notice to customers\n J. The money transfers were made without proper warning from financial institution\n K. The money transfers were made without proper notice to customers\n L. The money transfers were made without proper warning from financial institution\n M. The money transfers were made wi

In [43]:
response6=RAG.llm_usage(final6)
RAG.extract_answer(response6)

Device set to use cpu


"'people want to be able to transfer money internationally without any hassle and at a fair price'"